In [1]:
# Import necessary packages
from appgeopy import *
from my_packages import *

# --- Configuration Constants ---
# These constants define the dimensions and resolution of the output figures.
# Using constants makes it easier to change these values in one place.
FIGURE_WIDTH_PIXELS = 2000
FIGURE_HEIGHT_PIXELS = 1120
FIGURE_DPI = 300


def create_plot(
    axis,
    original_data,
    modeled_data,
    predicted_data,
    original_future,
    station_name,
    layer_num,
):
    """
    This function creates and configures the plot for a single station.

    Args:
        axis (matplotlib.axes.Axes): The matplotlib axis object to draw the plot on.
        original_data (pd.Series): The original monthly compaction data.
        modeled_data (pd.Series): The modeled compaction data.
        predicted_data (pd.Series): The predicted future compaction data.
        station_name (str): The name of the station being plotted.
        layer_num (str): The layer number being plotted.
    """
    # Plot the original monthly compaction data
    axis.plot(
        original_data,
        marker="s",
        markevery=3,
        ms=5,
        linestyle="-",
        # mfc="none",
        # mec="k",
        lw=2,
        color="darkgrey",
        label="Monthly MLCW",
        alpha=0.5,
    )

    axis.plot(
        original_future,
        marker="s",
        markevery=3,
        ms=5,
        linestyle="-",
        # mfc="none",
        # mec="k",
        lw=2,
        color="cornflowerblue",
        # label="Monthly MLCW",
        alpha=0.5,
    )

    # Plot the modeled compaction data
    axis.plot(
        modeled_data,
        marker="o",
        markevery=3,
        ms=5,
        linestyle="--",
        mfc="white",
        mec="k",
        lw=3,
        color="magenta",
        label="Modeled MLCW",
    )

    # Plot the predicted compaction data
    axis.plot(
        predicted_data,
        marker="D",
        markevery=3,
        ms=5,
        linestyle="--",
        mfc="white",
        mec="k",
        lw=3,
        color="lime",
        label="Predicted MLCW",
    )

    # Configure the appearance of the axis
    visualize.configure_axis(
        ax=axis,
        title=f"{station_name} - Layer {layer_num}",
        ylabel="Cumulative\nCompaction (mm)",
        hide_spines=["top", "right"],
        tick_direction="out",
        fontsize_base=14,
        major_tick_length=10,
        minor_tick_length=7,
    )

    # Configure the ticks on the y-axis
    # visualize.configure_ticks(ax=axis, y_major_interval=5)
    visualize.configure_datetime_ticks(
        ax=axis,
        major_interval=12,
        minor_interval=3,
        start_date=datetime(2016, 1, 1),
        end_date=datetime(2025, 1, 1),
        fontsize=14,
    )

    # Configure the legend
    visualize.configure_legend(
        ax=axis,
        labelspacing=0.1,
        handletextpad=0.3,
        scaling_factor=0.7,
        markerscale=1,
    )

In [2]:
"""
This is the main function of the script.|

It finds the relevant data file, processes the data for each station,
and generates a plot comparing original, modeled, and predicted
compaction data.
"""

# Find all CSV files that contain future and predicted data
data_files = glob("*Future_and_Predicted*.csv")

# Select the first file found
# selected_file = data_files[0]
for selected_file in tqdm(data_files[:]):

    # Extract the layer number from the filename
    layer_number = selected_file.split(".")[0].split("_")[-1]

    # Create a directory to save the output figures
    figure_save_directory = f"predicted_mlcw\Layer_{layer_number}"
    if not os.path.exists(figure_save_directory):
        os.makedirs(figure_save_directory)

    # Read the data from the CSV file and convert the 'time' column to datetime objects
    data = pd.read_csv(selected_file)
    data["time"] = pd.to_datetime(data["time"])

    # Get a list of all unique station names
    unique_stations = data["STATION"].unique()

    # Loop through each station and generate a plot
    for station in tqdm(unique_stations[:]):
        # Filter the data for the current station
        station_data = data.query("STATION==@station")
        station_data = station_data.set_index("time")

        # Separate the data into original, modeled, and predicted series
        original_compaction = station_data[f"MLCW_Layer_{layer_number}"]

        modeled_compaction = station_data["yhat"]
        # modeled_compaction = modeled_compaction - modeled_compaction.iloc[0]

        predicted_compaction = station_data["pred_MLCW"]
        # predicted_compaction = predicted_compaction - predicted_compaction.iloc[0]
        predicted_compaction = predicted_compaction.loc[
            modeled_compaction.last_valid_index() :
        ]

        original_compaction_future = original_compaction.loc[
            modeled_compaction.last_valid_index() :
        ]

        if (
            original_compaction.last_valid_index()
            != modeled_compaction.last_valid_index()
        ):
            original_compaction = original_compaction.loc[
                : modeled_compaction.last_valid_index()
            ]

            # Create the plot
            figure = plt.figure(figsize=(12, 4))
            axis = figure.add_subplot(111)

            create_plot(
                axis,
                original_compaction,
                modeled_compaction,
                predicted_compaction,
                original_compaction_future,
                station,
                layer_number,
            )

            # Auto-format the x-axis labels for better readability
            figure.autofmt_xdate(ha="right", rotation=30)
            figure.tight_layout()

            # Save the figure to the output directory
            visualize.save_figure_with_exact_dimensions(
                fig=figure,
                savepath=os.path.join(figure_save_directory, f"{station}.png"),
                width_px=FIGURE_WIDTH_PIXELS,
                height_px=FIGURE_HEIGHT_PIXELS,
                dpi=FIGURE_DPI,
            )

            # Close the plot to free up memory
            plt.close()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]